In [ ]:
! python3 -m pip install  "sep_plot @ git+https://github.com/SEP-software/sep-plot.git@6331a1e36d8e3cdb4cfbc3539f31bdad1eb465a7" 


In [ ]:
import numpy as np
from sep_plot import Grey
from sep_python import Hypercube, DoubleVector, get_sep_vector, default_io

io=default_io
vecA=io.vector_from_storage("../data/midpt.HH")
vec=get_sep_vector(vecA.get_hyper(),data_type="float64")
vec.get_nd_array()[:,:]=vecA.get_nd_array()[:,:]
vec.scale(100./15278435.0)
import holoviews as hv
hv.extension('bokeh','matplotlib')
Grey(vec)

In [ ]:
from generic_solver._pyOperator import Operator
from sep_python import DoubleVector
import numba
from math import sqrt
class slowFit(Operator):
    
    def __init__(self,slow,midpt):
        if not isinstance(slow,DoubleVector) or not isinstance(midpt,DoubleVector):
            raise Exception("Expecting DoubleVector for model and data")
        super().__init__(slow,midpt)
        self._hs=slow.get_hyper()
        self._hm=midpt.get_hyper()
    
    def forward(self,add,model,data):
        #if not self.checkDomainRange(model,data):
        #    raise Exception("Model and data do not match initializaiton")
        if not add:
            data.zero()
        m=model.get_nd_array()
        d=data.get_nd_array()
        slow_forward(m,d,self._hs.axes[0].o,self._hs.axes[0].d,self._hs.axes[1].o,
            self._hs.axes[1].d,self._hm.axes[0].o,self._hm.axes[0].d, self._hm.axes[1].o,
            self._hm.axes[1].d)
        
    def adjoint(self,add,model,data):
        #if not self.checkDomainRange(model,data):
        #    raise Exception("Model and data do not match initializaiton")
        if not add:
            model.zero()
        m=model.get_nd_array()
        d=data.get_nd_array()
        slow_adjoint(m,d,self._hs.axes[0].o,self._hs.axes[0].d,self._hs.axes[1].o,
            self._hs.axes[1].d,self._hm.axes[0].o,self._hm.axes[0].d, self._hm.axes[1].o,
            self._hm.axes[1].d)       
        
@numba.njit()
def slow_forward(m,d,ot_s,dt_s,os,ds,ot_m,dt_m,oo,od):
    for i_s in range(m.shape[0]):
        slow=os+ds*i_s
        slowSq=slow*slow
        for i_t in range(m.shape[1]):
            t0=ot_s+dt_s*i_t
            t0Sq=t0*t0
            for i_o in range(d.shape[0]):
                off=oo+i_o*od
                offSq=off*off
                timeSq=t0Sq+offSq*slowSq
                time=sqrt(timeSq)
                fT=(time-ot_m)/dt_m
                iT=int(fT)
                if iT < d.shape[1]-1:
                    fT=fT-iT
                    d[i_o,iT]+=m[i_s,i_t]*(1.-fT)
                    d[i_o,iT+1]+=m[i_s,i_t]*fT
@numba.njit()
def slow_adjoint(m,d,ot_s,dt_s,os,ds,ot_m,dt_m,oo,od):
    for i_s in range(m.shape[0]):
        slow=os+ds*i_s
        slowSq=slow*slow
        for i_t in range(m.shape[1]):
            t0=ot_s+dt_s*i_t
            t0Sq=t0*t0
            for i_o in range(d.shape[0]):
                off=oo+i_o*od
                offSq=off*off
                timeSq=t0Sq+offSq*slowSq
                time=sqrt(timeSq)
                fT=(time-ot_m)/dt_m
                iT=int(fT)
                if iT < d.shape[1]-1:
                    fT=fT-iT
                    m[i_s,i_t]+=d[i_o,iT]*(1.-fT)+fT*d[i_o,iT+1]

               

In [ ]:
from genericSolver.pyProblem import ProblemL2Linear,ProblemL2LinearReg
from genericSolver.pyLinearSolver import LCGsolver
from genericSolver.pyStopper import BasicStopper
from sep_python.hypercube import Hypercube,Axis
from sep_python.sep_vector import get_sep_vector
hyperD=vec.get_hyper()
axisS=Axis(n=31,o=.30,d=0.015)
hyperS=Hypercube([hyperD.axes[0],axisS])
slow=get_sep_vector(hyperS,data_type="float64")
data=vec.clone()
slowOp=slowFit(slow,data)
probL2=ProblemL2Linear(slow,data,slowOp)
stop=BasicStopper(niter=100)
solve=LCGsolver(stop)
solve.run(probL2)

In [ ]:
Grey(probL2.model)

In [ ]:
from genericSolver.pyProblem import ProblemL1Lasso,ProblemLinearReg
from genericSolver.pyNonLinearSolver import NLCGsolver
from genericSolver.pyStopper import BasicStopper
from sep_python.hypercube import Hypercube,Axis
from sep_python.sep_vector import get_sep_vector
from genericSolver.pySparseSolver import ISTAsolver, SplitBregmanSolver

hyperD=vec.get_hyper()
axisS=Axis(n=64,o=.30,d=0.015)
hyperS=Hypercube([hyperD.axes[0],axisS])
slow=get_sep_vector(hyperS,data_type="float64")
data=vec.clone()
slowOp=slowFit(slow,data)
probL1=ProblemL1Lasso(slow,data,slowOp,lambda_value=.1)
stop=BasicStopper(niter=100)
solve=ISTAsolver(stop)
solve.run(probL1,verbose=True)

In [ ]:
Grey(probL1.model,pclip=100)

In [ ]:
Grey(probL2.model,pclip=100)

In [ ]:
from scipy.signal import hilbert, chirp
from sep_python.sep_vector import get_sep_vector
def calc_envelope(inA):
    outA=inA.clone()
    mod=inA.get_nd_array()
    out=outA.get_nd_array()
    for i in range(mod.shape[0]):
        analytic_signal = hilbert(mod[i,:])
        out[i,:] = np.abs(analytic_signal)
    return outA

In [ ]:
outer=np.ndarray((vec.shape[1],),dtype=np.complex64)
inner=np.ndarray((vec.shape[1],),dtype=np.complex64)
axes=slow.get_hyper().axes
for i in range(outer.shape[0]):
    outer[i]=complex((32-.035*i+.00002*i*i)*axes[1].d+axes[1].o,axes[0].o+axes[0].d*i)
    inner[i]=complex((22-.032*i+.000014*i*i)*axes[1].d+axes[1].o,axes[0].o+axes[0].d*i)
#print(outer)

In [ ]:
from sep_plot import Graph
envL2=calc_envelope(probL2.model)
envL1=calc_envelope(probL1.model)
Grey(envL1,pclip=100)*Graph(outer)*Graph(inner)

In [ ]:
def find_rms_weight(panel,inner,outer):
    rms=get_sep_vector(ns=[inner.shape[0]],data_type="float64")
    wt=rms.clone()
    r=rms.get_nd_array()
    w=wt.get_nd_array()
    os=panel.get_hyper().axes[1].o
    ds=panel.get_hyper().axes[1].d
    slow=panel.get_nd_array()
    for i in range(r.shape[0]):
        b=max(0,int((inner[i].real-os)/ds))
        e=int((outer[i].real-os)/ds+.9)+1
        mx=slow[b,e]
        im=b
        for i_s in range(b+1,e):
            if slow[i_s,i] >mx:
                mx=slow[i_s,i]
                im=b
        r[i]=1./(os+ds*im)
        w[i]=mx
    return rms,wt


        

In [ ]:

class causal(Operator):
    
    def __init__(self,inA,outA):
        if not isinstance(inA,DoubleVector) or not isinstance(outA,DoubleVector):
            raise Exception("Expecting DoubleVector for model and data")
        super().__init__(inA,outA)

    
    def forward(self,add,model,data):
        #if not self.checkDomainRange(model,data):
        #    raise Exception("Model and data do not match initializaiton")
        if not add:
            data.zero()
        m=model.get_nd_array()
        d=data.get_nd_array()
        causal_forward(m,d)

    def adjoint(self,add,model,data):
        #if not self.checkDomainRange(model,data):
        #    raise Exception("Model and data do not match initializaiton")
        if not add:
            model.zero()
        m=model.get_nd_array()
        d=data.get_nd_array()
        causal_adjoint(m,d) 
        
@numba.njit()
def causal_forward(m,d):
    tot=0
    for i in range(m.shape[0]):
        tot=m[i]+tot
        d[i]+=tot
        
@numba.njit()
def causal_adjoint(m,d):
    tot=0
    for i in range(m.shape[0]-1,-1,-1):
        tot=d[i]+tot
        m[i]+=tot
        

In [ ]:

import copy
class Jop(Operator):

  def __init__(self,mod:DoubleVector,dat:DoubleVector,sc):
    if not mod.checkSame(dat):
        raise Exception("Model and data must be the same")
    
    self._sc=np.ravel(sc)
    #if mod.get_hyper().get_n123()!=self._sc.shape[0]:
    #    raise Exception(f"Expecting mask {mod.shape[0]} and model {mod.get_hyper().get_n123()} to be same size ")

    super().__init__(mod,dat)

  def forward(self,add,mod,dat):
   # self.checkDomainRange(mod,dat)
    if not add:
      dat.zero()
    d=np.ravel(dat.get_nd_array())
    m=np.ravel(mod.get_nd_array())
    d[:]+=m[:]*self._sc[:]

  def adjoint(self,add,mod,dat):
  #  self.checkDomainRange(mod,dat)
    if not add:
      mod.zero()
    d=np.ravel(dat.get_nd_array())
    m=np.ravel(mod.get_nd_array())
    m[:]+=d[:]*self._sc[:]
  

In [ ]:
def smooth_rms(rms):
    rout=rms.clone()
    r=rms.get_nd_array()
    o=rout.get_nd_array()
    for i in range(o.shape[0]):
        tot=0
        for ib in range(i-20,i+20):
            tot+=r[max(0,min(o.shape[0]-1,ib))]
            o[i]=tot/41
    return rout
rms,wt=find_rms_weight(envL1,inner,outer)
rms=smooth_rms(rms)

In [ ]:
Graph(rms)

In [ ]:
import math
from genericSolver.pyOperator import ChainOperator

def rms_to_data(rms):
    data=rms.clone()
    d=data.get_nd_array()
    r=rms.get_nd_array()
    for i in range(d.shape[0]):
        d[i]=r[i]*r[i]*(i+1)
    return data

def update_wt(wt):
    wtNew=wt.clone()
    w=wt.get_nd_array()
    o=wtNew.get_nd_array()
    for i in range(w.shape[0]):
        o[i]=w[i]*1/(i+1)
        #o[i]=1/(i+1)
        #o[i]=1
    return wtNew

def model_to_interval(model):
    interval=model.clone()
    m=model.get_nd_array()
    iv=interval.get_nd_array()
    for i in range(m.shape[0]):
        if m[i]>=0:
            iv[i]=math.sqrt(m[i])
        else:
            iv[i]=0
    return interval

def new_prob_wt(oper,wt,data):
    wt_op=Jop(data,data,wt.get_nd_array())
    new_data=wt.clone()
    wt_op.forward(False,data,new_data)
    new_op=ChainOperator(oper,wt_op)
    return new_op,new_data

In [ ]:
from genericSolver.pyProblem import ProblemL2Linear,ProblemL2LinearReg
from genericSolver.pyLinearSolver import LCGsolver
from genericSolver.pyStopper import BasicStopper
from sep_python.hypercube import Hypercube,Axis
from sep_python.sep_vector import get_sep_vector
r=rms.clone()
#r.get_nd_array()[:]=rms.get_nd_array()[0]
data=rms_to_data(r)
wtUse=update_wt(wt)
model=data.clone()
mask=rms.clone()
mask.get_nd_array()[:]=1
mask.get_nd_array()[0]=0
model.get_nd_array()[1:]=0
causal_op=causal(model,data)
op_wt,data_new=new_prob_wt(causal_op,wtUse,data)

probL2Dix=ProblemL2Linear(model,data_new,op_wt,grad_mask=mask)
stop=BasicStopper(niter=1000)
solve=LCGsolver(stop)
solve.run(probL2Dix,verbose=True)


In [ ]:
Graph(model_to_interval(probL2Dix.model))


In [ ]:
from genericSolver.pyProblem import ProblemL2Linear,ProblemL2LinearReg
from genericSolver.pyLinearSolver import LCGsolver
from genericSolver.pyStopper import BasicStopper
from sep_python.hypercube import Hypercube,Axis
from sep_python.sep_vector import get_sep_vector

data=rms_to_data(rms)
wtUse=update_wt(wt)
model=rms.clone()
mask=rms.clone()
mask.get_nd_array()[:]=1
mask.get_nd_array()[0]=0
model.get_nd_array()[1:]=0
causal_op=causal(model,data)
causal_op2=causal(model,data)
causal_combo=ChainOperator(causal_op,causal_op2)
op_wt,data_new=new_prob_wt(causal_combo,wtUse,data)
probL2Dix=ProblemL2LinearReg(model,data_new,op_wt,epsilon=100,grad_mask=mask)
stop=BasicStopper(niter=500)
solve=LCGsolver(stop)
solve.run(probL2Dix)

In [ ]:
vint2=rms.clone()
causal_op.forward(False,probL2Dix.model,vint2)
vint=model_to_interval(vint2)
Graph(vint)

In [ ]:
data=rms_to_data(rms)
wtUse=update_wt(wt)
model=rms.clone()
mask=rms.clone()
mask.get_nd_array()[:]=0
mask.get_nd_array()[100::100]=1

model.get_nd_array()[1:]=0
causal_op=causal(model,data)
causal_op2=causal(model,data)
causal_combo=ChainOperator(causal_op,causal_op2)
op_wt,data_new=new_prob_wt(causal_combo,wtUse,data)
probL2Dix=ProblemL2LinearReg(model,data_new,op_wt,epsilon=100,grad_mask=mask)
stop=BasicStopper(niter=500)
solve=LCGsolver(stop)
solve.run(probL2Dix)

In [ ]:
vint2=rms.clone()
causal_op.forward(False,probL2Dix.model,vint2)
vint=model_to_interval(vint2)
Graph(vint)+Graph(probL2Dix.model)

In [ ]:
data=rms_to_data(rms)
wtUse=update_wt(wt)
model=rms.clone()
mask=rms.clone()
mask.get_nd_array()[:]=0
mask.get_nd_array()[1::100]=1

model.get_nd_array()[1:]=0
causal_op=causal(model,data)
causal_op2=causal(model,data)
causal_op3=causal(model,data)

causal_combo1=ChainOperator(causal_op,causal_op2)
causal_combo=ChainOperator(causal_op3,causal_combo1)

op_wt,data_new=new_prob_wt(causal_combo,wtUse,data)
probL2Dix=ProblemL2LinearReg(model,data_new,op_wt,epsilon=100,grad_mask=mask)
stop=BasicStopper(niter=500)
solve=LCGsolver(stop)
solve.run(probL2Dix,verbose=True)

In [ ]:
vint2=rms.clone()
causal_combo1.forward(False,probL2Dix.model,vint2)
vint=model_to_interval(vint2)
Graph(vint)+Graph(probL2Dix.model)

In [ ]:
class Iop(Operator):

  def __init__(self,mod:DoubleVector,dat:DoubleVector):
    if not mod.checkSame(dat):
        raise Exception("Model and data must be the same")
    
 
    super().__init__(mod,dat)

  def forward(self,add,mod,dat):
   # self.checkDomainRange(mod,dat)
    if not add:
      dat.zero()
    d=np.ravel(dat.get_nd_array())
    m=np.ravel(mod.get_nd_array())
    d[:]+=m[:]

  def adjoint(self,add,mod,dat):
  #  self.checkDomainRange(mod,dat)
    if not add:
      mod.zero()
    d=np.ravel(dat.get_nd_array())
    m=np.ravel(mod.get_nd_array())
    m[:]+=d[:]

In [ ]:
from genericSolver.pySparseSolver import ISTCsolver
from genericSolver.pyProblem import ProblemLinearReg
data=rms_to_data(rms)
wtUse=update_wt(wt)
model=rms.clone()
mask=rms.clone()
mask.get_nd_array()[:]=1
#mask.get_nd_array()[0]=0

model.get_nd_array()[1:]=0
causal_op=causal(model,data)
causal_op2=causal(model,data)
causal_op3=causal(model,data)

causal_combo1=ChainOperator(causal_op,causal_op2)
#causal_combo=ChainOperator(causal_op3,causal_combo)
i_op=Iop(model,model)
op_wt,data_new=new_prob_wt(causal_combo1,wtUse,data)
probL1=ProblemLinearReg(model,data_new,op_wt,regsL1=[i_op],epsL1=[100.])
stop=BasicStopper(niter=1000)
solve=SplitBregmanSolver(stop)
solve.run(probL1)

In [ ]:
vint2=rms.clone()
causal_op=causal(model,data)

causal_op.forward(False,probL1.model,vint2)
vint=model_to_interval(vint2)
Graph(vint)
#print(type(probL1.model),type(vint2))